**Proposed use:**
Testing python package installation against AI Platform Training

In [ ]:
#auth notebook
export GOOGLE_APPLICATION_CREDENTIALS="PATH"

In [12]:
#define args & filenames
JOB_NAME_PREFIX=""
PIP_PKG_NAME_LIST=("pkg1","pkg2","pkg3")
JOB_DIR="gs://BUCKET_NAME/PATH"
RUNTIME_VERSION=""
PYTHON_VERSION=""
REGION=""

In [13]:
%%bash
mkdir -p trainer
touch trainer/__init__.py

In [14]:
import os, time

t = time.localtime()
t_str = time.strftime("%H%M%S", t)
JOB_NAME=JOB_NAME_PREFIX + t_str

if os.path.exists('requirements.txt'):
  os.remove('requirements.txt')
if os.path.exists('trainer/' + 'train.py'):
  os.remove('trainer/' + 'train.py')
with open('requirements.txt', 'a') as reqfile, open('trainer/' + 'train.py', 'a') as trainfile:
    for index in range(0,len(PIP_PKG_NAME_LIST)):
        reqfile.write(PIP_PKG_NAME_LIST[index])
        trainfile.write("import" + " " + PIP_PKG_NAME_LIST[index])
        if index < len(PIP_PKG_NAME_LIST) - 1:
            reqfile.write("\n")
            trainfile.write("\n")
reqfile.close()

In [15]:
%%writefile -a trainer/train.py


if __name__ == '__main__':
   
    print("hello world!")

Appending to trainer/train.py


In [11]:
pip install -r requirements.txt

ERROR: Could not find a version that satisfies the requirement pkg1
ERROR: No matching distribution found for pkg1
Note: you may need to restart the kernel to use updated packages.


In [19]:
!gcloud ai-platform local train \
  --package-path trainer \
  --module-name trainer.train \
  --job-dir local-training-output

hello world!


To take a quick anonymous survey, run:
  $ gcloud survey



In [29]:
!gcloud ai-platform jobs submit training $JOB_NAME \
  --job-dir $JOB_DIR \
  --package-path trainer \
  --module-name trainer.train \
  --runtime-version $RUNTIME_VERSION \
  --python-version $PYTHON_VERSION \
  --region $REGION

Job [fasttext_171142] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ai-platform jobs describe fasttext_171142

or continue streaming the logs with the command

  $ gcloud ai-platform jobs stream-logs fasttext_171142
jobId: fasttext_171142
state: QUEUED
